<a href="https://colab.research.google.com/github/Aiswariyat/audio-transcription-project/blob/main/Audio_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets librosa torchaudio soundfile


In [5]:
from datasets import load_dataset

# Load the minds14 dataset
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:5%]")




In [6]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch

def preprocess_audio(batch):
    audio_array, sample_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    audio_array = resampler(audio_array).squeeze().numpy()
    input_values = processor(audio_array, sampling_rate=16000, return_tensors="pt").input_values[0]
    batch["input_values"] = input_values
    return batch



In [10]:
def transcribe(batch):
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    batch["transcription"] = processor.decode(predicted_ids[0])
    return batch


In [11]:

dataset = dataset.map(preprocess_audio)
transcriptions = dataset.map(transcribe)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [13]:
with open("all_transcriptions.txt", "w") as f:
    for idx, item in enumerate(transcriptions):
        f.write(f"Transcription {idx + 1}:\n")
        f.write(item["transcription"] + "\n\n")
